# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal

Loaded lib <WinDLL 'C:\Users\lucbe\OneDrive\Documents\Bridge\robot\dds.dll', handle 7ffba2700000 at 0x1528b050820>


In [2]:
models = Models.load('../models')   # loading neural networks

INFO:tensorflow:Restoring parameters from ../models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ../models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ../models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ../models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ../models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ../models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ../models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ../models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

In [4]:
hand = 'Q432.KJ654.4.432'
auction = ['PAD_START', '2N', 'PASS','3C','PASS','3D','PASS'] # Here, it wrongly passes on the stayman answer, just like he would pass a 3D opening
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)
bid = bot_bid.bid(auction)

print(bid.samples) #The samples are correct
print(bid.bid)
print(bid.to_dict()['candidates'])

['xx.9x.AKJ9xx.T98 AK9.A8x.QT.AKJxx JT8x.QTx.8xxx.Qx Qxxx.KJxxx.x.xxx', 'KJT.Q8x.9xx.QJ9x A8x.Ax.AQTx.AKT8 9xx.T9x.KJ8xx.xx Qxxx.KJxxx.x.xxx', 'K98x.Txx.Q9xxx.x Jx.AQx.AKJTx.KQ9 ATx.98.8x.AJT8xx Qxxx.KJxxx.x.xxx', 'Jxxx.T9x.JTxx.Tx AT8.A8.AK8x.AQ8x K9.Qxx.Q9xx.KJ9x Qxxx.KJxxx.x.xxx', 'J98xx.x.T9xxx.K8 AKT.A98.K8x.AQTx x.QTxx.AQJx.J9xx Qxxx.KJxxx.x.xxx', 'T8x.9x.Jxx.QT98x K9x.Axx.AQ8x.AKx AJx.QT8.KT9xx.Jx Qxxx.KJxxx.x.xxx', 'A9x.Qxx.T98x.Jxx KJ.A9.AQJxx.KQT8 T8xx.T8x.Kxx.A9x Qxxx.KJxxx.x.xxx', '9xx.A9x.QJT9x.Q9 AKT.Tx.AK8x.AKTx J8x.Q8x.xxx.J8xx Qxxx.KJxxx.x.xxx', 'T8xx.Q9x.QT9x.Tx AJ.Ax.AJ8xx.AK8x K9x.T8x.Kxx.QJ9x Qxxx.KJxxx.x.xxx', '9xx.QTx.8xx.98xx ATx.A9.ATx.AKQTx KJ8.8xx.KQJ9xx.J Qxxx.KJxxx.x.xxx']
PASS
[{'call': 'PASS', 'insta_score': 0.9976871013641357}]


In [5]:
hand = 'T873.AK6.KJ32.43'
auction = ['PASS','PASS','1H','2H','3H'] # Here, it correctly bids 4S
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)
bid = bot_bid.bid(auction)

print(bid.samples) #The samples are correct
print(bid.bid)
print(bid.to_dict()['candidates'])

['xx.T8xxx.9x.K98x T8xx.AKx.KJxx.xx x.QJ9xx.A8.AQJTx AKQJ9x..QTxxx.xx', 'KJ.T9x.QT8xx.9xx T8xx.AKx.KJxx.xx x.QJ8xxx.A9x.AQx AQ9xxx.x.x.KJT8x', 'AJxx.Txxx.Qxx.xx T8xx.AKx.KJxx.xx .QJ98x.A8xx.QJ98 KQ9xx.x.T9.AKTxx', 'Qx.J98xx.A8.Qxxx T8xx.AKx.KJxx.xx Ax.QTxxx.QTxx.A8 KJ9xx..9xx.KJT9x', 'KQx.Txxx.T98.Q8x T8xx.AKx.KJxx.xx 9.QJ98x.AQxx.Axx AJxxx.x.xx.KJT9x', 'Q9.QTxx.A9.T9xxx T8xx.AKx.KJxx.xx KJ.J9xxx.Tx.AKJ8 Axxxx.8.Q8xxx.Qx', 'x.Txx.Q98xx.AQxx T8xx.AKx.KJxx.xx AJx.QJ98xxx.AT.T KQ9xx..xx.KJ98xx', '9.JT9.Txx.KJ98xx T8xx.AKx.KJxx.xx AKx.Q8xxxx.A.Qxx QJxxx.x.Q98xx.AT', 'QJx.QJT9x.T8x.T9 T8xx.AKx.KJxx.xx K.8xxxx.Q9x.AKQJ A9xxx..Axx.8xxxx', 'J.JTxx.Q8xx.Axxx T8xx.AKx.KJxx.xx AQx.Q98xxx.xx.KQ K9xxx..AT9.JT98x']
4S
[{'call': '4S', 'insta_score': 0.7305068373680115}]


In [6]:
hand = 'T873.AK6.KJ32.43'
auction = ['1H','PASS','PASS','2H','PASS'] #Here, it wrongly fit the artificial 3H
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)
bid = bot_bid.bid(auction)

print(bid.samples) #The samples are correct
print(bid.bid)
print(bid.to_dict()['candidates'])

['KJ.J98xx.x.AJTxx T8xx.AKx.KJxx.xx xxx.QTx.98x.Q98x AQ9x.xx.AQTxx.Kx', 'Q.JTxxx.AT9x.AJ9 T8xx.AKx.KJxx.xx 9xxx.Q8x.8xx.8xx AKJx.9x.Qx.KQTxx', 'AQx.Q9xxx.Q.KT8x T8xx.AKx.KJxx.xx J9.JT8xx.Txx.Qxx Kxxx..A98xx.AJ9x', 'x.Q98xxx.Axx.AQ9 T8xx.AKx.KJxx.xx xx.JTx.Q8xx.Txxx AKQJ9x.x.T9.KJ8x', 'KQ.QJ8xx.xx.AKTx T8xx.AKx.KJxx.xx 9xx.Txx.xx.J9xxx AJxx.9x.AQT98.Q8', 'AQ.JTxxxx.A8x.Q8 T8xx.AKx.KJxx.xx xx.Q8x.Q9xx.T9xx KJ9xx.9.Tx.AKJxx', 'Ax.Q8xxx.Ax.Axxx T8xx.AKx.KJxx.xx 9xx.J9xx.T98x.Q9 KQJx.T.Qxx.KJT8x', 'K9x.QJ8xxx.9.KQJ T8xx.AKx.KJxx.xx Q.T9x.Q8xx.9xxxx AJxxx.x.ATxx.AT8', 'AJx.Q8xxx.8x.AKx T8xx.AKx.KJxx.xx .JT9x.9x.QJ98xxx KQ9xxx.x.AQTxx.T', 'x.QJT8xxx.Qx.AK8 T8xx.AKx.KJxx.xx Jxxx.9x.A9.Txxxx AKQ9.x.T8xxx.QJ9']
3H
[{'call': '3H', 'insta_score': 0.31888464093208313, 'expected_score': 379.02323359257014}, {'call': '4H', 'insta_score': 0.2546205520629883, 'expected_score': 327.2922176247282}, {'call': 'PASS', 'insta_score': 0.3088884651660919, 'expected_score': 34.91254548361309}]


In [7]:
hand = 'AQ752.KJ92.K.AKQ'
auction = ["PASS","1N","PASS"] #Here, it correctly does a stayman
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)
bid = bot_bid.bid(auction)

print(bid.samples) #The samples are correct
print(bid.bid)
print(bid.to_dict()['candidates'])

['JTx.Tx.JTxx.Jxxx Kx.AQ8x.AQ9xx.Tx 98x.xxx.8xx.98xx AQxxx.KJ9x.K.AKQ', '8xx.Tx.QT8xxx.9x KT.AQx.AJ9x.J8xx J9x.8xxx.xx.Txxx AQxxx.KJ9x.K.AKQ', '9x.T8xx.T8x.xxxx KT8x.AQx.AQxx.Jx Jx.xx.J9xxx.T98x AQxxx.KJ9x.K.AKQ', 'xx.8xxx.JTxx.8xx KJ98x.AQx.AQ9.9x T.Tx.8xxxx.JTxxx AQxxx.KJ9x.K.AKQ', 'T8xx.xx.Q8xx.8xx KJ.AQTx.A9xx.JTx 9x.8xx.JTxx.9xxx AQxxx.KJ9x.K.AKQ', 'Txx.xx.T8xxx.xxx KJ9.AT8x.AQJ9.xx 8x.Qxx.xxx.JT98x AQxxx.KJ9x.K.AKQ', 'Jx.Txx.T9xxx.xxx KT9x.AQ.AQxx.8xx 8x.8xxx.J8x.JT9x AQxxx.KJ9x.K.AKQ', 'T9x.8xxx.98.J98x K8xx.AQ.AQJx.Txx J.Txx.Txxxxx.xxx AQxxx.KJ9x.K.AKQ', 'xx.Qx.T9xxx.9xxx KJT.ATxxx.AQ8.J8 98x.8x.Jxxx.Txxx AQxxx.KJ9x.K.AKQ', 'T9x.Tx.Qxxx.9xxx KJx.AQ8.AT98.Jxx 8x.xxxx.Jxxx.T8x AQxxx.KJ9x.K.AKQ']
2C
[{'call': '2C', 'insta_score': 0.940109372138977}]


In [8]:
hand = 'AQ752.KJ92.K.AKQ'
auction = [ 'PASS','PASS','PASS','1S',"PASS","1N","PASS"] #Here, it bids 2C ("stayaman !") instead of 3H, like in the first hand of the demo (https://lorserker.github.io/ben/demo/viz.html?deal=1). Note that the insta score is exactly the same as the previous example
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)
bid = bot_bid.bid(auction)

print(bid.samples) #The samples are correct
print(bid.bid)
print(bid.to_dict()['candidates'])

['K98x.Txx.8x.98xx Jx.A8x.QTxxxx.Jx Tx.Qxx.AJ9x.Txxx AQxxx.KJ9x.K.AKQ', 'T98.xx.A9xx.JTxx Kx.AQx.JT8x.8xxx Jxx.T8xx.Qxxx.9x AQxxx.KJ9x.K.AKQ', 'Kxxx.xx.QJTx.Txx 9.AQx.8xxx.J8xxx JT8.T8xx.A9xx.9x AQxxx.KJ9x.K.AKQ', 'KT.QT.QJ9xx.JT9x 8xx.8xx.AT8xxx.x J9x.Axxx.x.8xxxx AQxxx.KJ9x.K.AKQ', 'T98x.AQx.T8xx.Tx .Txxx.AQ9xxx.9xx KJxx.8x.Jx.J8xxx AQxxx.KJ9x.K.AKQ', 'JT8.QTxx.AQ8x.xx K9x.x.Txx.T98xxx xx.A8xx.J9xxx.Jx AQxxx.KJ9x.K.AKQ', 'KJTx.Tx.Txxx.9xx 9.Q8x.AJxxx.T8xx 8xx.Axxx.Q98.Jxx AQxxx.KJ9x.K.AKQ', 'K9x.A8.Q9xx.9xxx Txx.Txxx.AT8x.Tx J8.Qxx.Jxxx.J8xx AQxxx.KJ9x.K.AKQ', 'KJ8x.Qxx.98x.Jxx T.AT8xx.QTxxx.Tx 9xx.x.AJxx.98xxx AQxxx.KJ9x.K.AKQ', 'K98.Qxxx.Jxx.T8x Jxx.8xx.A98xx.xx Tx.AT.QTxx.J9xxx AQxxx.KJ9x.K.AKQ']
2C
[{'call': '2C', 'insta_score': 0.940109372138977}]
